## Урок 8. Вебинар “Производная функции нескольких переменных. Часть 1

#### Найти частные производные первого и второго порядка. Убедиться в равенстве смешанных производных.

> 1. $U = x^3 + 3xy^2 + z^2 - 39x - 36y + 2z + 26$

$$U'_x = 3x^2 + 3y^2 - 39, \quad  U'_y = 6xy - 36, \quad U'_z = 2z + 2$$
$$U''_{xx} = 6x, \quad  U''_{yx} = 6y, \quad U''_{zx} = 0$$
$$U''_{xy} = 6y, \quad  U''_{yy} = 6x, \quad U''_{zy} = 0$$
$$U''_{xz} = 0, \quad  U''_{yz} = 0, \quad U''_{zz} = 2$$
$$U''_{xy} = U''_{yx} = 6y, \quad U''_{xz} = U''_{zx} = 0, \quad U''_{yz} = U''_{zy} = 0$$

> 2. $U=\frac{256}{x}+\frac{x^{2}}{y}+\frac{y^{2}}{z}+z^{2}$

$$U'_x = -\frac{256}{x^2}+\frac{2x}{y}, \quad U'_y = -\frac{x^2}{y^2}+\frac{2y}{z}, \quad U'_z = -\frac{y^2}{z^2}+2z$$
$$U''_{xx} = \frac{512}{x^3}+\frac{2}{y}, \quad  U''_{yx} = -\frac{2x}{y^2}, \quad U''_{zx} = 0$$
$$U''_{xy} = -\frac{2x}{y^2}, \quad  U''_{yy} = \frac{2x^2}{y^3}+\frac{2}{z}, \quad U''_{zy} = -\frac{2y}{z^2}$$
$$U''_{xz} = 0, \quad  U''_{yz} = -\frac{2y}{z^2}, \quad U''_{zz} = \frac{2y^2}{z^3}+2$$
$$U''_{xy} = U''_{yx} = -\frac{2x}{y^2}, \quad U''_{xz} = U''_{zx} = 0, \quad U''_{yz} = U''_{zy} = -\frac{2y}{z^2}$$

#### Найти производную функции:

> 3. $U = x^2 + y^2 + z^2$ по направлению вектора $\vec{c}(-9,8,-12)$ в точку $M(8 ;-12 ; 9)$.

$$|\vec{c}|=\sqrt{x_{0}^{2}+y_{0}^{2}+z_{0}^{2}}=\sqrt{(-9)^{2}+8^{2}+(-12)^{2}}=\sqrt{81+64+144}=\sqrt{289}=17$$
$$\vec{c_{0}}=\frac{\vec{c}}{|\vec{c}|}=\left(-\frac{9}{17},\frac{8}{17},-\frac{12}{17}\right)$$
$$U'_{x}=2x, \quad U'_{y}=2y, \quad U'_{z}=2z$$
$$\operatorname{grad}\left.U\right|_{M}=\operatorname{grad}\left.U\right|_{(8,-12,9)}=(2\cdot8, -2\cdot12, 2\cdot9)=(16, -24, 18)$$
$$\left.U_{\vec{c}}'\right|_{M}=\left.U_{\vec{c}}'\right|_{(8,-12,9)}=-\frac{9}{17} \cdot 16 - \frac{8}{17} \cdot 24 - \frac{12}{17} \cdot 18=\frac{-144-192-216}{17}=-\frac{552}{17}$$

> 4. $U=e^{x^{2}+y^{2}+z^{2}}$ по направлению вектора $\vec{d}=(4,-13,-16)$ в точку $L(-16 ; 4 ;-13)$.

$$|\vec{d}|=\sqrt{x_{0}^{2}+y_{0}^{2}+z_{0}^{2}}=\sqrt{4^{2}+(-13)^{2}+(-16)^{2}}=\sqrt{16+169+256}=\sqrt{441}=21$$
$$\vec{d_{0}}=\frac{\vec{d}}{|\vec{d}|}=\left(\frac{4}{21},-\frac{13}{21},-\frac{16}{21}\right)$$
$$U'_{x}=2x \cdot e^{x^{2}+y^{2}+z^{2}}, \quad U'_{y}=2y \cdot e^{x^{2}+y^{2}+z^{2}}, \quad U'_{z}=2z \cdot e^{x^{2}+y^{2}+z^{2}}$$
$$\operatorname{grad}\left.U\right|_{L}=\operatorname{grad}\left.U\right|_{(-16,4,-13)}=(-2 \cdot 16 \cdot e^{(-16)^{2}+4^{2}+(-13)^{2}}, 2 \cdot 4 \cdot e^{(-16)^{2}+4^{2}+(-13)^{2}}, -2 \cdot 13 \cdot e^{(-16)^{2}+4^{2}+(-13)^{2}})=(-32 \cdot e^{441}, 8 \cdot e^{441}, -8 \cdot e^{441})$$
$$\left.U_{\vec{d}}'\right|_{L}=\left.U_{\vec{d}}'\right|_{(-16,4,-13)}=-\frac{4}{21} \cdot 32 \cdot e^{441} - \frac{13}{21} \cdot 8 \cdot e^{441} + \frac{16}{21} \cdot 8 \cdot e^{441}=\frac{-128-104+128}{21} \cdot e^{441}=-\frac{104}{21} \cdot e^{441}$$

> 5. \* $U=\log _{21}\left(x^{2}+y^{2}+z^{2}\right)$ в точку $F(-19 ; 8 ;-4)$ по самому быстрому направлению.

#### Исследовать на экстремум следующие функции:

> 6. $U=x^{2} y+\frac{1}{3} y^{3}+2 x^{2}+3 y^{2}-1$

> 7. \* $U=e^{\frac{-x}{2}}(x^{2}+y^{2})$

8. ** __С помощью метода наименьших квадратов (МНК) подобрать значения параметров $a$ и $b$ для сигмоидальной функции $y=\frac{1}{1+e^{-a x+b}}$ по заданному распределению в файле «SGN.ipynb»__